In [38]:
import pandas as pd
from datetime import datetime
import holidays
from geopy import distance
from meteostat import Point, Daily
import sys
sys.path.append('../scripts/')

In [20]:
df=pd.read_csv('/home/b/Documents/week8/Logistic-Optimization/data/merged_data.csv',low_memory=False)

In [22]:
from geopy.distance import geodesic
sample = df.copy()
sample[["Trip_Origin_x", "Trip_Origin_y"]] = sample["Trip Origin"].str.strip(r"[()]").str.split(",", expand=True).astype(float)

In [25]:
print("Columns before dropping:", df.columns)

Columns before dropping: Index(['Trip ID', 'Trip Origin', 'Trip Destination', 'Trip Start Time',
       'Trip End Time', 'id', 'order_id', 'driver_id', 'driver_action', 'lat',
       'lng', 'created_at', 'updated_at'],
      dtype='object')


In [28]:
df['Trip Start Time']= pd.to_datetime(df['Trip Start Time'])
df['Trip End Time'] = pd.to_datetime(df['Trip End Time'])
merged_df = df.drop(["id"], axis=1)
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1557740 entries, 0 to 1557739
Data columns (total 12 columns):
 #   Column            Non-Null Count    Dtype         
---  ------            --------------    -----         
 0   Trip ID           536020 non-null   float64       
 1   Trip Origin       536020 non-null   object        
 2   Trip Destination  536020 non-null   object        
 3   Trip Start Time   534369 non-null   datetime64[ns]
 4   Trip End Time     536019 non-null   datetime64[ns]
 5   order_id          1557740 non-null  int64         
 6   driver_id         1557740 non-null  int64         
 7   driver_action     1557740 non-null  object        
 8   lat               1557740 non-null  float64       
 9   lng               1557740 non-null  float64       
 10  created_at        0 non-null        float64       
 11  updated_at        0 non-null        float64       
dtypes: datetime64[ns](2), float64(5), int64(2), object(3)
memory usage: 142.6+ MB


In [29]:
merged_df.head()

,Trip ID,Trip Origin,Trip Destination,Trip Start Time,Trip End Time,order_id,driver_id,driver_action,lat,lng,created_at,updated_at
0,391996.0,"6.508813001668548,3.37740316890347","6.650969799999999,3.3450307",2021-07-01 07:28:04,2021-07-01 07:29:37,392001,243828,accepted,6.602207,3.270465,NaN,NaN
1,391997.0,"6.4316714,3.4555375","6.4280814653326,3.4721885847586",2021-07-01 06:38:04,2021-07-01 07:07:28,392001,243588,rejected,6.592097,3.287445,NaN,NaN
2,391998.0,"6.631679399999999,3.3388976","6.508324099999999,3.3590397",2021-07-01 06:21:02,2021-07-01 07:02:23,392001,243830,rejected,6.596133,3.281784,NaN,NaN
3,391999.0,"6.572757200000001,3.3677082","6.584881099999999,3.3614073",2021-07-01 07:16:07,2021-07-01 07:29:42,392001,243539,rejected,6.596142,3.280526,NaN,NaN
4,392001.0,"6.6010417,3.2766339","6.4501069,3.3916154",2021-07-01 09:30:59,2021-07-01 09:34:36,392001,171653,rejected,6.609232,3.288800,NaN,NaN


### Data preparation

In [36]:
# create an object of class for data preparation
from data_preparation import DataPreparation
prep_class = DataPreparation()

In [ ]:
# adding hours column - Trip Start Time has some missing values
merged_df['start_hour'] = merged_df['Trip Start Time'].apply(lambda row: prep_class.get_hour_of_day(row))
merged_df['end_hour'] = merged_df['Trip End Time'].apply(lambda row: prep_class.get_hour_of_day(row))

In [ ]:
# adding distance travelled column 
merged_df['trip_distance_km'] = merged_df[['Trip Origin','Trip Destination']].apply(lambda row: prep_class.get_trip_distance(row[0], row[1]),axis=1)

In [37]:
# adding time taken to deliver order
merged_df['trip_time'] = merged_df[['Trip Start Time','Trip End Time']].apply(lambda row: prep_class.get_trip_duration(row[0], row[1]),axis=1)

In [39]:
y_min = merged_df['lat'].min()
y_max = merged_df['lat'].max()
x_min = merged_df['lng'].min()
x_max = merged_df['lng'].max()
# y = latitude, x = longitude

In [40]:
# Set time period i.e. from November, 2021 to May, 2022 : 187 days
start_date = datetime(2021, 11, 13)
end_date = datetime (2022, 5, 19)

# using map bounds as coordinates i.e. y_min, y_max, x_min, x_max
# y = latitude, x = longitude
avg_lat = (y_min+y_max)/2
avg_long = (x_min+x_max)/2
altitude = 41 # from google

# using nearest station instead of coordinates
# from : https://meteostat.net/en/station/65201?t=2022-10-09/2022-10-16

# Get daily data
data = Daily(65201, start_date, end_date)
data = data.fetch() # returns a dataframe

# get dates from datetime
merged_df['Start Date'] = merged_df['Trip Start Time'].apply(lambda x: x.date())

rainy_days = data['prcp'].dropna().index # DatetimeIndex object

# convert datetimeindex to list of datetime objects
rainy_list = list(rainy_days.to_pydatetime())
rainy_dates_list = [d.date() for d in rainy_list]
     

In [41]:
merged_df['rainy'] = merged_df['Start Date'].isin(rainy_dates_list)

In [42]:
# set holidays
nigeria_holidays = holidays.Nigeria
holiday_dates = []
for  k, v in nigeria_holidays(years = 2021).items():
    holiday_date = datetime(k.year, k.month, k.day)
    if(start_date<holiday_date<end_date):
        holiday_dates.append(k)
# 2022
for  k, v in nigeria_holidays(years = 2022).items():
    holiday_date = datetime(k.year, k.month, k.day)
    if(start_date<holiday_date<end_date):
        holiday_dates.append(k)

In [43]:
# converting to dataframe
merged_df['holiday'] = merged_df['Start Date'].isin(holiday_dates)
merged_df['holiday'].value_counts()

False    1551648
True        6092
Name: holiday, dtype: int64

In [47]:
from datetime import timedelta

weekend_date = []
def daterange(date1, date2):
    for n in range(int ((date2 - date1).days)+1):
        yield date1 + timedelta(n)

weekends = [5,6] # 0 based index for days of week. 5 -Saturday, 6 = Sunday
no_of_weekends = 0
for dt in daterange(start_date, end_date):
    if dt.weekday() in weekends:                
        weekend_date.append(dt.strftime("%Y-%m-%d"))

In [48]:
# convert to datetime
weekend_dates = [datetime.strptime(date_string, "%Y-%m-%d") for date_string in weekend_date]
weekend_dates_list = [d.date() for d in weekend_dates]

In [49]:
merged_df['weekend'] = merged_df['Start Date'].isin(weekend_dates_list)
merged_df['weekend'].value_counts()

False    1528065
True       29675
Name: weekend, dtype: int64

In [50]:

# removing holiday column because all values are false
# removing start date column because the values were only used for calculating 
# weekend, rainy days and holidays

merged_df.drop(['Start Date','holiday'], axis=1).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1557740 entries, 0 to 1557739
Data columns (total 15 columns):
 #   Column            Non-Null Count    Dtype          
---  ------            --------------    -----          
 0   Trip ID           536020 non-null   float64        
 1   Trip Origin       536020 non-null   object         
 2   Trip Destination  536020 non-null   object         
 3   Trip Start Time   534369 non-null   datetime64[ns] 
 4   Trip End Time     536019 non-null   datetime64[ns] 
 5   order_id          1557740 non-null  int64          
 6   driver_id         1557740 non-null  int64          
 7   driver_action     1557740 non-null  object         
 8   lat               1557740 non-null  float64        
 9   lng               1557740 non-null  float64        
 10  created_at        0 non-null        float64        
 11  updated_at        0 non-null        float64        
 12  trip_time         534368 non-null   timedelta64[ns]
 13  rainy             1557740 n

In [53]:
# saving as csv file
merged_df.to_csv('data_feat_eng.csv')